# 게임개발

현민이는 게임 캐릭터가 맵 안에서 움직이는 시스템을 개발 중이다. 캐릭터가 있는 장소는 1 X 1 크기의 정사각형으로 이뤄진 N X M 크기의 직사각형으로, 각각의 칸은 육지 또는 바다이다. 캐릭터는 동서남북 중 한 곳을 바라본다.

맵의 각 칸은 (A, B)로 나타낼 수 있고, A는 북쪽으로부터 떨어진 칸의 개수, B는 서쪽으로부터 떨어진 칸의 개수이다. 캐릭터는 상하좌우로 움직일 수 있고, 바다로 되어 있는 공간에는 갈 수 없다. 캐릭터의 움직임을 설정하기 위해 정해 놓은 매뉴얼은 이러하다.

현재 위치에서 현재 방향을 기준으로 왼쪽 방향(반시계 방향으로 90도 회전한 방향)부터 차례대로 갈 곳을 정한다.

캐릭터의 바로 왼쪽 방향에 아직 가보지 않은 칸이 존재한다면, 왼쪽 방향으로 횐전한 다음 왼쪽으로 한 칸을 전진한다. 왼쪽 방향에 가보지 않은 칸이 없다면, 왼쪽 방향으로 회전만 수행하고 1단계로 돌아간다.

만약 네 방향 모두 이미 가본 칸이거나 바다로 되어 있는 칸인 경우에는, 바라보는 방향을 유지한 채로 한 칸 뒤로 가고 1단계로 돌아간다. 단, 이때 뒤쪽 방향이 바다인 칸이라 뒤로 갈 수 없는 경우에는 움직임을 멈춘다.

현민이는 위 과정을 반복적으로 수행하면서 캐릭터의 움직임에 이상이 있는지 테스트하려고 한다. 메뉴얼에 따라 캐릭터를 이동시킨 뒤에, 캐릭터가 방문한 칸의 수를 출력하는 프로그램을 만드시오.

### input
첫째 줄에 맵의 세로 크기 N과 가로 크기 M을 공백으로 구분하여 입력한다.
(3 <= N, M <= 50)

둘째 줄에 게임 캐릭터가 있는 칸의 좌표 (A, B)와 바라보는 방햔 d가 각각 서로 공백으로 구분하여 주어진다. 방향 d의 값으로는 다음과 같이 4가지가 존재한다.

0 : 북쪽
1 : 동쪽
2 : 남쪽
3 : 서쪽

셋째 줄부터 맵이 육지인지 바다인지에 대한 정보가 주어진다. N개의 줄에 맵의 상태가 북쪽부터 남쪽 순서대로, 각 줄의 데이터는 서쪽부터 동쪽 순서대로 주어진다. 맵의 외각은 항상 바다로 되어 있다.

0 : 육지
1 : 바다
처음에 게임 캐릭터가 위치한 칸의 상태는 항상 육지이다.

### output
첫째 줄에 이동을 마친 후 캐릭터가 방문한 칸의 수를 출력한다.

In [41]:
from time import sleep
import random
import uuid
from datetime import datetime


def make_test_case() -> str:
    """ 테스트 케이스 생성기 """
    MIN_N, MAX_N = 3, 50
    MIN_M, MAX_M = 3, 50
    DIRECTION_TYPES = 4

    N = random.randrange(MIN_N, MAX_N + 1)
    M = random.randrange(MIN_M, MAX_N + 1)
    DIRECTION = random.randrange(0, DIRECTION_TYPES)

    start_list = []
    BOARD = []
    for row in range(N):
        tmp_row = []
        for col in range(M):
            map_status = random.randrange(0, 1 + 1)
            if map_status:
                start_list.append((row, col))
            tmp_row.append(map_status)
        BOARD.append(tmp_row)

    START_ROW, START_COL = random.choice(start_list)

    text = (
               f"{' '.join(map(str, [N, M]))}\n"
               f"{' '.join(map(str, [START_ROW, START_COL]))} {DIRECTION}\n"
           ) + ''.join([f"{' '.join(map(str, row))}\n" for row in BOARD])

    with open(file=f"./test/{uuid.uuid4()}.txt", mode='w', encoding="utf-8") as f:
        f.write(text)

    print("Test case creation complete!")



# make_test_case()
# 에러를 발생하는 렌덤 케이스가 있어서 실행안함

Test case creation complete!
Test case creation complete!
Test case creation complete!
Test case creation complete!
Test case creation complete!
Test case creation complete!
Test case creation complete!
Test case creation complete!
Test case creation complete!
Test case creation complete!
Test case creation complete!
Test case creation complete!
Test case creation complete!
Test case creation complete!
Test case creation complete!
Test case creation complete!
Test case creation complete!
Test case creation complete!
Test case creation complete!
Test case creation complete!


In [46]:
import _io
from algorithm_util import time_check


@time_check
def solution_0(test: _io.TextIOWrapper) -> int:
    """ 풀이 정답 """
    SEA = 0
    GROUND = 1
    TURN = 4
    MOVE = [
        (-1, 0),  # 북
        (0, 1),  # 동
        (1, 0),  # 남
        (0, -1),  # 서
    ]

    BACK = [
        (1, 0),  # 남
        (0, -1),  # 서
        (-1, 0),  # 북
        (0, 1),  # 동
    ]

    CHANGE_DIRECTIONS = {
        0: 3,  # 북 > 서
        3: 2,  # 서 > 남
        2: 1,  # 남 > 동
        1: 0,  # 동 > 북
    }

    ROW, COL = map(int, test.readline().split())
    crow, ccol, cdirection = map(int, test.readline().split())
    board = [list(map(int, test.readline().split())) for _ in range(ROW)]
    visited = [[False] * COL for _ in range(ROW)]

    answer = 1
    while True:

        can_go = False
        visited[crow][ccol] = True
        for _ in range(TURN):
            cdirection = CHANGE_DIRECTIONS[cdirection]
            trow, tcol = MOVE[cdirection]
            nrow, ncol = trow + crow, tcol + ccol
            if not (0 <= nrow < ROW and 0 <= ncol < COL):
                continue
            if board[nrow][ncol]:
                continue
            if visited[nrow][ncol]:
                continue
            answer += 1
            can_go = True
            crow, ccol = nrow, ncol
            break

        if not can_go:  # 갈 수 없으면
            trow, tcol = BACK[cdirection]
            nrow, ncol = trow + crow, tcol + ccol
            if not (0 <= nrow < ROW and 0 <= ncol < COL):
                break
            if board[nrow][ncol]:
                break
            crow, ccol = nrow, ncol

    return answer

In [47]:
from algorithm_util import result_df

solutions = [
    answer,
    solution_0,
]

result_df(solutions)

,solution_n,time,result,test_file_path
0,ansmwer,0.0,3,test\original.txt
1,1,0.0,3,test\original.txt
